In [66]:
import pandas as pd

df_fr = pd.read_excel(r"../data/financial_reports/xml/2020.xlsx")
df_fr.head()

,start,end,today,heading,g_NS_info/region,g_NS_info/reporting_ns_name,g_NS_info/report_date,g_NS_info/ccy,g_NS_info/reporting_ccy,g_NS_info/g_contact1/contact1_name,...,g_exp_ns/g_exp_breakdown/exp_health_total_calc,g_exp_ns/g_exp_breakdown/calc_note_exp_socioec,g_exp_ns/g_exp_breakdown/calc_note_exp_ns_inst,g_exp_ns/realloc,g_exp_ns/realloc_amt,_id,_uuid,_submission_time,_validation_status,_index
0,2020-05-12T08:39:06.843-05:00,2020-05-12T15:46:14.892-05:00,2020-05-11,NaN,Americas,Paraguay,2020-05-12,CHF,CHF,Ing. Carlos Escobar Goiburu,...,NaN,NaN,NaN,NaN,NaN,91507.0,d752c0bc-e683-440e-85ec-ee541856c97e,2020-05-12T20:46:15,validation_status_approved,1.0
1,2020-05-13T12:51:38.500+05:30,2020-05-13T14:43:44.263+05:30,2020-05-13,NaN,MENA,"Iran, Islamic Rep.",2020-05-12,USD,USD,Mansooreh Bagheri,...,NaN,NaN,NaN,NaN,NaN,91685.0,ebfbe264-de9e-4dfa-a518-94b20a576e16,2020-05-13T10:23:15,validation_status_approved,2.0
2,2020-05-13T10:52:14.296-00:00,2020-05-13T11:39:23.312-00:00,2020-05-13,NaN,Africa,"Gambia, The",2020-05-13,local,GMD,Alasan Senghore,...,NaN,NaN,NaN,NaN,NaN,91699.0,7478e911-c96c-4540-bd80-8296eb976642,2020-05-13T11:39:24,validation_status_approved,3.0
3,2020-05-13T10:21:58.909-04:00,2020-06-24T12:05:15.964+02:00,2020-05-12,NaN,Americas,Dominica,2020-05-13,CHF,CHF,Sandra Charter-Rolle,...,NaN,NaN,NaN,NaN,NaN,91770.0,e685241a-a8ac-419b-9ef5-277f5921ae94,2020-05-13T14:59:17,validation_status_approved,4.0
4,2020-05-13T12:48:12.568+03:00,2020-05-28T09:43:34.382+02:00,2020-05-13,NaN,Europe,Finland,2020-05-13,EUR,EUR,Marko Korhonen,...,NaN,NaN,NaN,NaN,NaN,91776.0,dfee2f98-d020-497c-baa6-54d419985b98,2020-05-13T15:16:20,validation_status_approved,5.0


In [67]:
# keep most recent per country
df_fr['end'] = pd.to_datetime(df_fr['end'])
df_fr_ = pd.DataFrame()
for country in df_fr['g_NS_info/reporting_ns_name'].unique():
    df_fr_last = df_fr[df_fr['g_NS_info/reporting_ns_name']==country] 
    df_fr_last = df_fr_last[df_fr_last['end']==df_fr_last['end'].max()]
    if len(df_fr_last)>1:
        print(df_fr_last.head())
    df_fr_ = df_fr_.append(df_fr_last, ignore_index=True)
print(len(df_fr_), len(df_fr), df_fr['g_NS_info/reporting_ns_name'].nunique())

178 543 178


In [68]:
# select only approved reports
df_fr_ = df_fr_[df_fr_["_validation_status"] == "validation_status_approved"]

In [69]:
# convert currency
from datetime import date
from currency_converter import CurrencyConverter
currConv = CurrencyConverter()

def convertCurrency(row):
    
    if row['g_NS_info/reporting_ccy'] == 'XCD':
        new = row['g_ns_income/inc_otherns'] * 0.34
    elif row['g_NS_info/reporting_ccy'] == 'SZL':
        new = row['g_ns_income/inc_otherns'] * 0.064
    elif row['g_NS_info/reporting_ccy'] == 'GEL':
        new = row['g_ns_income/inc_otherns'] * 0.27
    elif row['g_NS_info/reporting_ccy'] == 'BWP':
        new = row['g_ns_income/inc_otherns'] * 0.085
    elif row['g_NS_info/reporting_ccy'] == 'NAD':
        new = row['g_ns_income/inc_otherns'] * 0.062
    elif row['g_NS_info/reporting_ccy'] == 'BOB':
        new = row['g_ns_income/inc_otherns'] * 0.13
    elif row['g_NS_info/reporting_ccy'] == 'XOF':
        new = row['g_ns_income/inc_otherns'] * 0.0017
    elif row['g_NS_info/reporting_ccy'] == 'ARS':
        new = row['g_ns_income/inc_otherns'] * 0.0098
    elif row['g_NS_info/reporting_ccy'] == 'BSD':
        new = row['g_ns_income/inc_otherns'] * 0.92
    elif row['g_NS_info/reporting_ccy'] == 'BBD':
        new = row['g_ns_income/inc_otherns'] * 0.45
    elif row['g_NS_info/reporting_ccy'] == 'CRC':
        new = row['g_ns_income/inc_otherns'] * 0.0015
    elif row['g_NS_info/reporting_ccy'] == 'TTD':
        new = row['g_ns_income/inc_otherns'] * 0.13
    elif row['g_NS_info/reporting_ccy'] == 'SDG':
        new = row['g_ns_income/inc_otherns'] * 0.0024
    elif row['g_NS_info/reporting_ccy'] == 'FJD':
        new = row['g_ns_income/inc_otherns'] * 0.45
    elif row['g_NS_info/reporting_ccy'] == 'SBD':
        new = row['g_ns_income/inc_otherns'] * 0.11
    elif row['g_NS_info/reporting_ccy'] == 'MZN':
        new = row['g_ns_income/inc_otherns'] * 0.016
    elif row['g_NS_info/reporting_ccy'] == 'XAF':
        new = row['g_ns_income/inc_otherns'] * 0.0017
    elif row['g_NS_info/reporting_ccy'] == 'KZT':
        new = row['g_ns_income/inc_otherns'] * 0.0021
    elif row['g_NS_info/reporting_ccy'] == 'GTQ':
        new = row['g_ns_income/inc_otherns'] * 0.12
    elif row['g_NS_info/reporting_ccy'] == 'MRU':
        new = row['g_ns_income/inc_otherns'] * 0.025
    elif row['g_NS_info/reporting_ccy'] == 'CLP':
        new = row['g_ns_income/inc_otherns'] * 0.0013
    elif row['g_NS_info/reporting_ccy'] == 'RSD':
        new = row['g_ns_income/inc_otherns'] * 0.0094
    elif row['g_NS_info/reporting_ccy'] == 'MVR':
        new = row['g_ns_income/inc_otherns'] * 0.059
    elif row['g_NS_info/reporting_ccy'] == 'MUR':
        new = row['g_ns_income/inc_otherns'] * 0.023
    elif row['g_NS_info/reporting_ccy'] == 'STD':
        new = row['g_ns_income/inc_otherns'] * 0.000044
    elif row['g_NS_info/reporting_ccy'] == 'UYU':
        new = row['g_ns_income/inc_otherns'] * 0.021
    elif row['g_NS_info/reporting_ccy'] == 'SLL':
        new = row['g_ns_income/inc_otherns'] * 0.00009
    elif row['g_NS_info/reporting_ccy'] == 'RWF':
        new = row['g_ns_income/inc_otherns'] * 0.00092
    elif row['g_NS_info/reporting_ccy'] == 'TZS':
        new = row['g_ns_income/inc_otherns'] * 0.0004
    elif row['g_NS_info/reporting_ccy'] == 'IQD':
        new = row['g_ns_income/inc_otherns'] * 0.00063
    elif row['g_NS_info/reporting_ccy'] == 'LYD':
        new = row['g_ns_income/inc_otherns'] * 0.2
    elif row['g_NS_info/reporting_ccy'] == 'DJF':
        new = row['g_ns_income/inc_otherns'] * 0.0051
    elif row['g_NS_info/reporting_ccy'] == 'ETB':
        new = row['g_ns_income/inc_otherns'] * 0.022
    elif row['g_NS_info/reporting_ccy'] == 'SCR':
        new = row['g_ns_income/inc_otherns'] * 0.065
    elif row['g_NS_info/reporting_ccy'] == 'BND':
        new = row['g_ns_income/inc_otherns'] * 0.69
    elif row['g_NS_info/reporting_ccy'] != 'CHF':
        try:
            new = currConv.convert(row['g_ns_income/inc_otherns'], row['g_NS_info/reporting_ccy'], 'CHF', date=date(2020, 12, 31))
        except:
            print('ERROR:', row['g_NS_info/reporting_ccy'], 'NOT FOUND!!!')
            new = 0
    else:
        new = int(row['g_ns_income/inc_otherns']) if not pd.isna(row['g_ns_income/inc_otherns']) else 0
    return new

df_fr_['g_ns_income/inc_otherns'] = df_fr_.apply(convertCurrency, axis=1)
df_fr_.head()

,start,end,today,heading,g_NS_info/region,g_NS_info/reporting_ns_name,g_NS_info/report_date,g_NS_info/ccy,g_NS_info/reporting_ccy,g_NS_info/g_contact1/contact1_name,...,g_exp_ns/g_exp_breakdown/exp_health_total_calc,g_exp_ns/g_exp_breakdown/calc_note_exp_socioec,g_exp_ns/g_exp_breakdown/calc_note_exp_ns_inst,g_exp_ns/realloc,g_exp_ns/realloc_amt,_id,_uuid,_submission_time,_validation_status,_index
0,2020-11-17T14:57:49.954-03:00,2020-11-17 15:44:05.759000-03:00,2020-11-16,NaN,Americas,Paraguay,2020-11-15,USD,USD,Judith Molinas,...,NaN,NaN,NaN,NaN,NaN,195918.0,6336a93b-c129-42df-8bfc-4ffd405d936c,2020-11-17T18:44:07,validation_status_approved,427.0
1,2020-05-13T12:51:38.500+05:30,2020-05-13 14:43:44.263000+05:30,2020-05-13,NaN,MENA,"Iran, Islamic Rep.",2020-05-12,USD,USD,Mansooreh Bagheri,...,NaN,NaN,NaN,NaN,NaN,91685.0,ebfbe264-de9e-4dfa-a518-94b20a576e16,2020-05-13T10:23:15,validation_status_approved,2.0
3,2020-12-03T15:08:14.271-04:00,2020-12-04 10:53:02.100000-05:00,2020-12-02,NaN,Americas,Dominica,2020-11-15,local,XCD,Sylvester Jno Baptiste,...,NaN,NaN,NaN,NaN,NaN,204612.0,53a0fd22-ba26-4f87-8d6b-cc36456e715d,2020-12-03T19:23:39,validation_status_approved,516.0
4,2020-08-19T11:22:08.034+03:00,2020-08-25 10:07:00.838000+02:00,2020-08-19,NaN,Europe,Finland,2020-08-19,EUR,EUR,Marko Korhonen,...,NaN,NaN,NaN,NaN,NaN,159589.0,5ce28af9-eadd-4c3d-aaaf-c653afd1aa58,2020-08-19T15:55:02,validation_status_approved,322.0
5,2020-11-09T12:09:22.131+01:00,2020-11-09 12:29:47.190000+01:00,2020-11-09,NaN,Europe,Poland,2020-03-04,local,PLN,Magdalena Stefańska,...,NaN,NaN,NaN,NaN,NaN,190821.0,34d2067f-ee7c-4cd7-8d0a-a9e0f4722f69,2020-11-09T11:29:41,validation_status_approved,364.0


In [108]:
# correct for reporting time 
df_fr = df_fr_.copy()
df_fr['g_financial_data/reporting_start'] = pd.to_datetime(df_fr['g_financial_data/reporting_start'])
df_fr['g_financial_data/reporting_end'] = pd.to_datetime(df_fr['g_financial_data/reporting_end'])
df_fr['diff'] = df_fr['g_financial_data/reporting_end'] - df_fr['g_financial_data/reporting_start']
df_fr['diff'] = abs(df_fr['diff'].dt.days)
df_fr['diff'] = (df_fr['diff'] - 366)/366
df_fr['g_ns_income/inc_otherns'] = df_fr['g_ns_income/inc_otherns'] * 1/(1+df_fr['diff'])
df_fr.drop(columns=['diff'], inplace=True)

In [109]:
# keep only relevant columns
df_fr = df_fr[['g_NS_info/reporting_ns_name', 'g_ns_income/inc_otherns']]
df_fr['g_ns_income/inc_otherns'].fillna(0, inplace=True)
df_fr.head()

,g_NS_info/reporting_ns_name,g_ns_income/inc_otherns
0,Paraguay,325252.553119
1,"Iran, Islamic Rep.",0.000000
3,Dominica,0.000000
4,Finland,0.000000
5,Poland,0.000000


In [110]:
# calculate average income
print(df_fr['g_ns_income/inc_otherns'].mean(), df_fr[df_fr['g_ns_income/inc_otherns']>0]['g_ns_income/inc_otherns'].mean())

274001.59797181614 599626.6854165832


In [111]:
# compare to FDRS 2019
df_fdrs = pd.read_excel(r'../data/FDRS_2019.xlsx')
df_fdrs['KPI_incomeFromNSsLC_CHF'].fillna(0, inplace=True)
print(df_fdrs['KPI_incomeFromNSsLC_CHF'].mean(), df_fdrs[df_fdrs['KPI_incomeFromNSsLC_CHF']>0]['KPI_incomeFromNSsLC_CHF'].mean())

756061.737804878 1653255.0
